In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nba_api.stats.static import teams

def prepare_upcoming_games_data(season_games_csv):
    # Load season games data
    data = pd.read_csv(season_games_csv)
    
    # Process home and away data
    home_data = data[['DATE', 'Start (ET)', 'Home/Neutral']].copy()
    home_data['Home_Away'] = 'Home'
    home_data['MATCHUP'] = home_data['Home/Neutral'] + ' vs. ' + data['Visitor/Neutral']
    home_data.rename(columns={'Home/Neutral': 'Team'}, inplace=True)
    home_data['WL_encoded'] = np.nan
    
    away_data = data[['DATE', 'Start (ET)', 'Visitor/Neutral']].copy()
    away_data['Home_Away'] = 'Away'
    away_data['MATCHUP'] = away_data['Visitor/Neutral'] + ' @ ' + home_data['Team']  # Adjusted to use '@' for away games
    away_data.rename(columns={'Visitor/Neutral': 'Team'}, inplace=True)
    away_data['WL_encoded'] = np.nan
    
    final_data = pd.concat([home_data, away_data], ignore_index=True)
    final_data.sort_values(by=['DATE', 'Start (ET)', 'Home_Away'], inplace=True)
    final_data.reset_index(drop=True, inplace=True)
    
    # Convert 'DATE' column to datetime format
    final_data['DATE'] = pd.to_datetime(final_data['DATE'], format='%a, %b %d, %Y')
    
    # Get unique team information from the NBA API
    teams_info = teams.get_teams()
    teams_df = pd.DataFrame(teams_info)
    teams_df.rename(columns={'id': 'TEAM_ID', 'full_name': 'TEAM_NAME', 'abbreviation': 'TEAM_ABBREVIATION'}, inplace=True)
    
    # Merge final_data with teams_df to include TEAM_ID and abbreviations
    final_data = pd.merge(final_data, teams_df[['TEAM_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION']], left_on='Team', right_on='TEAM_NAME', how='left')
    
    # Ensure all team names in MATCHUP are abbreviations
    for index, row in final_data.iterrows():
        for _, team_row in teams_df.iterrows():
            final_data.at[index, 'MATCHUP'] = final_data.at[index, 'MATCHUP'].replace(team_row['TEAM_NAME'], team_row['TEAM_ABBREVIATION'])
    
    # Extract and filter for upcoming games
    today = pd.Timestamp.now().floor('D')  # Normalize to avoid time part
    week_out = today + timedelta(days=7)
    upcoming_games = final_data[(final_data['DATE'] >= today) & (final_data['DATE'] <= week_out)]
    upcoming_games.sort_values(by='DATE', inplace=True)
    upcoming_games.reset_index(drop=True, inplace=True)
    
    # Format the 'DATE' column to match the example output's 'GAME_DATE' format
    upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
    
    # Correct the column name for consistency
    upcoming_games.rename(columns={'Home_Away': 'home_away'}, inplace=True)
    
    # Drop unnecessary columns and adjust to match the target dataset structure
    upcoming_games = upcoming_games[['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_ID', 'TEAM_NAME']]
    
    # Create OPPOSING_TEAM column
    upcoming_games['OPPOSING_TEAM'] = np.nan  # Placeholder for opposing team names
    
    # Populate TEAM_NAME and OPPOSING_TEAM with full names
    for index, row in upcoming_games.iterrows():
        home_team, away_team = row['MATCHUP'].split(' @ ') if row['home_away'] == 'Away' else row['MATCHUP'].split(' vs. ')
        home_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == home_team]['TEAM_NAME'].values[0]
        away_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == away_team]['TEAM_NAME'].values[0]
        
        if row['home_away'] == 'Home':
            upcoming_games.at[index, 'TEAM_NAME'] = home_team_full_name
            upcoming_games.at[index, 'OPPOSING_TEAM'] = away_team_full_name
        else:
            upcoming_games.at[index, 'TEAM_NAME'] = away_team_full_name
            upcoming_games.at[index, 'OPPOSING_TEAM'] = home_team_full_name
    
    
    return upcoming_games

# Example usage
season_games_csv = 'data/23_24_season_games.csv'
upcoming_games_df = prepare_upcoming_games_data(season_games_csv)
print(upcoming_games_df.head())


    GAME_DATE      MATCHUP home_away     TEAM_ID       TEAM_NAME  \
0  2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   
1  2024-02-28  DEN vs. SAC      Home  1610612743  Denver Nuggets   
2  2024-02-28    SAC @ DEN      Away  1610612758  Denver Nuggets   
3  2024-02-28  CHI vs. CLE      Home  1610612741   Chicago Bulls   
4  2024-02-28    CLE @ CHI      Away  1610612739   Chicago Bulls   

          OPPOSING_TEAM  
0  New Orleans Pelicans  
1      Sacramento Kings  
2      Sacramento Kings  
3   Cleveland Cavaliers  
4   Cleveland Cavaliers  


C:\Users\ghadf\AppData\Local\Temp\ipykernel_23532\3454788115.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games.sort_values(by='DATE', inplace=True)
C:\Users\ghadf\AppData\Local\Temp\ipykernel_23532\3454788115.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
C:\Users\ghadf\AppData\Local\Temp\ipykernel_23532\3454788115.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nba_api.stats.static import teams

def prepare_upcoming_games_data(season_games_csv, player_game_logs_csv, expand_with_players=False):
    # Load season games data
    data = pd.read_csv(season_games_csv)
    
    # Process home and away data
    home_data = data[['DATE', 'Start (ET)', 'Home/Neutral']].copy()
    home_data['Home_Away'] = 'Home'
    home_data['MATCHUP'] = home_data['Home/Neutral'] + ' vs. ' + data['Visitor/Neutral']
    home_data.rename(columns={'Home/Neutral': 'Team'}, inplace=True)
    home_data['WL_encoded'] = np.nan
    
    away_data = data[['DATE', 'Start (ET)', 'Visitor/Neutral']].copy()
    away_data['Home_Away'] = 'Away'
    away_data['MATCHUP'] = away_data['Visitor/Neutral'] + ' @ ' + home_data['Team']  # Adjusted to use '@' for away games
    away_data.rename(columns={'Visitor/Neutral': 'Team'}, inplace=True)
    away_data['WL_encoded'] = np.nan
    
    final_data = pd.concat([home_data, away_data], ignore_index=True)
    final_data.sort_values(by=['DATE', 'Start (ET)', 'Home_Away'], inplace=True)
    final_data.reset_index(drop=True, inplace=True)
    
    # Convert 'DATE' column to datetime format
    final_data['DATE'] = pd.to_datetime(final_data['DATE'], format='%a, %b %d, %Y')
    
    # Get unique team information from the NBA API
    teams_info = teams.get_teams()
    teams_df = pd.DataFrame(teams_info)
    teams_df.rename(columns={'id': 'TEAM_ID', 'full_name': 'TEAM_NAME', 'abbreviation': 'TEAM_ABBREVIATION'}, inplace=True)
    
    # Merge final_data with teams_df to include TEAM_ID and abbreviations
    final_data = pd.merge(final_data, teams_df[['TEAM_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION']], left_on='Team', right_on='TEAM_NAME', how='left')
    
    # Ensure all team names in MATCHUP are abbreviations
    for index, row in final_data.iterrows():
        for _, team_row in teams_df.iterrows():
            final_data.at[index, 'MATCHUP'] = final_data.at[index, 'MATCHUP'].replace(team_row['TEAM_NAME'], team_row['TEAM_ABBREVIATION'])
    
    # Extract and filter for upcoming games
    today = pd.Timestamp.now().floor('D')  # Normalize to avoid time part
    week_out = today + timedelta(days=7)
    upcoming_games = final_data[(final_data['DATE'] >= today) & (final_data['DATE'] <= week_out)]
    upcoming_games.sort_values(by='DATE', inplace=True)
    upcoming_games.reset_index(drop=True, inplace=True)
    
    # Format the 'DATE' column to match the example output's 'GAME_DATE' format
    upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
    
    # Correct the column name for consistency
    upcoming_games.rename(columns={'Home_Away': 'home_away'}, inplace=True)
    
    # Drop unnecessary columns and adjust to match the target dataset structure
    upcoming_games = upcoming_games[['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_ID', 'TEAM_NAME']]
    
    # Create OPPOSING_TEAM column
    upcoming_games['OPPOSING_TEAM'] = np.nan  # Placeholder for opposing team names
    
    # Populate TEAM_NAME and OPPOSING_TEAM with full names
    for index, row in upcoming_games.iterrows():
        home_team, away_team = row['MATCHUP'].split(' @ ') if row['home_away'] == 'Away' else row['MATCHUP'].split(' vs. ')
        home_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == home_team]['TEAM_NAME'].values[0]
        away_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == away_team]['TEAM_NAME'].values[0]
        
        if row['home_away'] == 'Home':
            upcoming_games.at[index, 'TEAM_NAME'] = home_team_full_name
            upcoming_games.at[index, 'OPPOSING_TEAM'] = away_team_full_name
        else:
            upcoming_games.at[index, 'TEAM_NAME'] = away_team_full_name
            upcoming_games.at[index, 'OPPOSING_TEAM'] = home_team_full_name
    
    

    # Load player game logs to use for fetching rosters
    player_game_logs = pd.read_csv(player_game_logs_csv)
    print(player_game_logs.columns)
    print(upcoming_games.columns)
    
    if expand_with_players:
        expanded_games_with_players = pd.DataFrame()

        # Assuming player_game_logs_csv is correctly loaded into player_game_logs DataFrame
        player_game_logs = pd.read_csv('data\player_game_logs_winr.csv')

        expanded_rows = []

        for _, game in upcoming_games.iterrows():
            team_name = game['TEAM_NAME']
            team_players = player_game_logs[player_game_logs['TEAM_NAME'] == team_name]

            for _, player in team_players.iterrows():
                expanded_row = game.copy().to_dict()
                expanded_row['Player_ID'] = player['Player_ID']
                expanded_row['PLAYER_NAME'] = player['PLAYER_NAME']
                expanded_rows.append(expanded_row)

        expanded_games_with_players = pd.DataFrame(expanded_rows)

        expanded_games_with_players = expanded_games_with_players.drop_duplicates(subset=['PLAYER_NAME'])


        # After expanding games with players, adjust columns to match player_game_logs_winr structure
        # Here you would set default values or transform columns as needed to match the structure
        # For demonstration, let's set some columns to default values
        expanded_games_with_players['SEASON_ID'] = '22023'
        expanded_games_with_players['MIN'] = np.nan  # Just an example, adjust according to your data needs
        
        # Return the expanded DataFrame
        return expanded_games_with_players


    return upcoming_games

# Example usage with file paths
season_games_csv = 'data/23_24_season_games.csv'
player_game_logs_csv = 'data/player_game_logs_winr.csv'
upcoming_games = prepare_upcoming_games_data(season_games_csv, player_game_logs_csv, expand_with_players=True)
print(upcoming_games.head())
print(len(upcoming_games))


C:\Users\ghadf\AppData\Local\Temp\ipykernel_24412\311170071.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games.sort_values(by='DATE', inplace=True)
C:\Users\ghadf\AppData\Local\Temp\ipykernel_24412\311170071.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
C:\Users\ghadf\AppData\Local\Temp\ipykernel_24412\311170071.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'OPPONENT_ABBREVIATION', 'TEAM_NAME',
       'OPPONENT_NAME', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE', 'home_away'],
      dtype='object')
Index(['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_ID', 'TEAM_NAME',
       'OPPOSING_TEAM'],
      dtype='object')
      GAME_DATE      MATCHUP home_away     TEAM_ID       TEAM_NAME  \
0    2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   
51   2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   
106  2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   
139  2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   
191  2024-02-28  IND vs. NOP      Home  1610612754  Indiana Pacers   

           

In [16]:
def load_data():
    data = pd.read_csv('data/player_game_logs_winr.csv')
    data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'])
    data.sort_values(by='GAME_DATE', inplace=True)
    return data

# Load the existing games data
previous_games = load_data()

# Ensure GAME_DATE is in datetime format for comparison
upcoming_games['GAME_DATE'] = pd.to_datetime(upcoming_games['GAME_DATE'])

# Filter out upcoming games that have dates already in previous games
unique_upcoming_games = upcoming_games[~upcoming_games['GAME_DATE'].isin(previous_games['GAME_DATE'])]

# Concatenate the unique upcoming games to the previous games dataset
concatenated_data = pd.concat([previous_games, unique_upcoming_games], ignore_index=True)

# Sort the concatenated data by GAME_DATE to maintain chronological order
concatenated_data.sort_values(by='GAME_DATE', inplace=True)

# Reset the index of the concatenated DataFrame
concatenated_data.reset_index(drop=True, inplace=True)

# Perform checks after concatenation
print("Number of rows in concatenated dataset:", len(concatenated_data))
print("Latest date in previous games dataset:", previous_games['GAME_DATE'].max())
print("Earliest date in upcoming games dataset:", unique_upcoming_games['GAME_DATE'].min())
print("Latest date in concatenated dataset:", concatenated_data['GAME_DATE'].max())

# Example usage
concat_data = concatenated_data
print(concat_data.head())
print(len(concat_data))


Number of rows in concatenated dataset: 8811
Latest date in previous games dataset: 2024-02-27 00:00:00
Earliest date in upcoming games dataset: 2024-02-28 00:00:00
Latest date in concatenated dataset: 2024-03-06 00:00:00
  SEASON_ID  Player_ID     Game_ID  GAME_DATE      MATCHUP WL   MIN  FGM  \
0     22023     203484  22300061.0 2023-10-24  DEN vs. LAL  W  36.0  8.0   
1     22023     203076  22300061.0 2023-10-24    LAL @ DEN  L  34.0  6.0   
2     22023    1630559  22300061.0 2023-10-24    LAL @ DEN  L  31.0  4.0   
3     22023     203952  22300062.0 2023-10-24  GSW vs. PHX  L  27.0  4.0   
4     22023    1630228  22300062.0 2023-10-24  GSW vs. PHX  L  20.0  4.0   

    FGA  FG_PCT  ...               PLAYER_NAME  TEAM_ABBREVIATION  \
0  12.0   0.667  ...  Kentavious Caldwell-Pope                DEN   
1  17.0   0.353  ...             Anthony Davis                LAL   
2  11.0   0.364  ...             Austin Reaves                LAL   
3  12.0   0.333  ...            Andrew Wiggin

In [2]:
import pandas as pd

# Assuming you've already read in your df1
unique_teams = pd.read_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\team_ids.csv')

# take only the first 
unique_teams = unique_teams.drop_duplicates(subset='TEAM_ID', keep='first')



#change team name "LA Clippers" to "Los Angeles Clippers"
unique_teams['TEAM_NAME'] = unique_teams['TEAM_NAME'].replace('LA Clippers', 'Los Angeles Clippers')

# get unique values from team_id and season_id columns
unique_team_ids = unique_teams['TEAM_ID'].unique()
print(unique_team_ids)
unique_team_names = unique_teams['TEAM_NAME'].unique()
print(unique_team_names)
unique_team_names = unique_teams['TEAM_ABBREVIATION'].unique()
print(unique_team_names)

#take only the columns team_id, team_name, season_id, and game_id

#print(df1.head())

# Assuming df1 has been read in and unique_teams has been created
team_to_abbreviation = dict(zip(unique_teams['TEAM_NAME'], unique_teams['TEAM_ABBREVIATION']))

print(team_to_abbreviation)


[1610612737 1610612738 1610612739 1610612740 1610612741 1610612742
 1610612743 1610612744 1610612745 1610612746 1610612747 1610612748
 1610612749 1610612750 1610612751 1610612752 1610612753 1610612754
 1610612755 1610612756 1610612757 1610612758 1610612759 1610612760
 1610612761 1610612762 1610612763 1610612764 1610612765 1610612766]
['Atlanta Hawks' 'Boston Celtics' 'Cleveland Cavaliers'
 'New Orleans Pelicans' 'Chicago Bulls' 'Dallas Mavericks'
 'Denver Nuggets' 'Golden State Warriors' 'Houston Rockets'
 'Los Angeles Clippers' 'Los Angeles Lakers' 'Miami Heat'
 'Milwaukee Bucks' 'Minnesota Timberwolves' 'Brooklyn Nets'
 'New York Knicks' 'Orlando Magic' 'Indiana Pacers' 'Philadelphia 76ers'
 'Phoenix Suns' 'Portland Trail Blazers' 'Sacramento Kings'
 'San Antonio Spurs' 'Oklahoma City Thunder' 'Toronto Raptors' 'Utah Jazz'
 'Memphis Grizzlies' 'Washington Wizards' 'Detroit Pistons'
 'Charlotte Hornets']
['ATL' 'BOS' 'CLE' 'NOP' 'CHI' 'DAL' 'DEN' 'GSW' 'HOU' 'LAC' 'LAL' 'MIA'
 'MIL' '

1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766

current_teams = [1610612739, 1610612737, 1610612738, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [3]:


# Convert 'Date' column to datetime format in final_data
final_data['DATE'] = pd.to_datetime(final_data['DATE'])

#count nan values
print(final_data.isnull().sum())
print(unique_teams.isnull().sum())

# Merge final_data with unique_teams on the team names to get the TEAM_ID. Use 'left' to ensure final_data size isn't increased.
final_data = final_data.merge(unique_teams, left_on='Team', right_on='TEAM_NAME', how='left')


#count nan values
final_data.isnull().sum()
#view the team_id columns that have nan values
final_data[final_data['TEAM_ID'].isnull()]

#print(final_data)

DATE             0
Start (ET)       0
Team             0
Home_Away        0
MATCHUP          0
WL_encoded    2400
dtype: int64
TEAM_ID              0
TEAM_NAME            0
TEAM_ABBREVIATION    0
dtype: int64


,DATE,Start (ET),Team,Home_Away,MATCHUP,WL_encoded,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION


In [4]:
# Create a function to extract both teams from the matchup string
def extract_teams(matchup):
    # Split the string using 'vs.' as the delimiter
    teams = matchup.split(' vs. ')
    return teams

# Apply the function to the 'MATCHUP' column
final_data['Home_Team'], final_data['Away_Team'] = zip(*final_data['MATCHUP'].map(extract_teams))

# Determine the opposing team for each row
final_data['Opposing_Team'] = final_data.apply(lambda row: row['Away_Team'] if row['Team'] == row['Home_Team'] else row['Home_Team'], axis=1)

# Create a mapping from the team name to the TEAM_ID using the unique_teams dataframe
team_to_id = dict(zip(unique_teams['TEAM_NAME'], unique_teams['TEAM_ID']))

# Map the opposing team name to its ID
final_data['TEAM_ID_OPP'] = final_data['Opposing_Team'].map(team_to_id)

# Drop the columns we created for the intermediate steps (optional)
final_data.drop(columns=['Home_Team', 'Away_Team', 'Opposing_Team'], inplace=True)
print(final_data['TEAM_ID'])
print(final_data.columns)

0       1610612740
1       1610612745
2       1610612744
3       1610612757
4       1610612762
           ...    
2395    1610612741
2396    1610612758
2397    1610612762
2398    1610612742
2399    1610612759
Name: TEAM_ID, Length: 2400, dtype: int64
Index(['DATE', 'Start (ET)', 'Team', 'Home_Away', 'MATCHUP', 'WL_encoded',
       'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_ID_OPP'],
      dtype='object')


In [5]:


# Convert TEAM_ID and SEASON_ID to integers
#final_data['TEAM_ID'] = final_data['TEAM_ID'].astype(int)
#final_data['SEASON_ID'] = final_data['SEASON_ID'].astype(int)

# If you want to convert them to strings after converting to integers, uncomment the following lines:
# final_data['TEAM_ID'] = final_data['TEAM_ID'].astype(str)
# final_data['SEASON_ID'] = final_data['SEASON_ID'].astype(str)

# Drop 'TEAM_NAME' and 'Start (ET)' column as it's redundant
final_data.drop(['TEAM_NAME', 'Start (ET)'], axis=1, inplace=True)

#final_data['TEAM_ID'] = final_data['TEAM_ID'].astype('int64')

# Assuming your dataframe is named 'final_data'
final_data['YEAR'] = pd.to_datetime(final_data['DATE']).dt.year
final_data['MONTH'] = pd.to_datetime(final_data['DATE']).dt.month
final_data['DAY'] = pd.to_datetime(final_data['DATE']).dt.day

# Dropping the original Date column and Team, Home_Away, Matchup columns
final_data.drop(['DATE', 'Team'], axis=1, inplace=True)

# add a matchup identifier column on when they happen on the same day
#final_data['GAME_ID'] = final_data.groupby(['MATCHUP','YEAR', 'MONTH', 'DAY']).ngroup()

print(final_data)


     Home_Away                                         MATCHUP  WL_encoded  \
0         Away  Golden State Warriors vs. New Orleans Pelicans         NaN   
1         Away      Portland Trail Blazers vs. Houston Rockets         NaN   
2         Home  Golden State Warriors vs. New Orleans Pelicans         NaN   
3         Home      Portland Trail Blazers vs. Houston Rockets         NaN   
4         Away              Los Angeles Clippers vs. Utah Jazz         NaN   
...        ...                                             ...         ...   
2395      Home         Chicago Bulls vs. Oklahoma City Thunder         NaN   
2396      Away                  Utah Jazz vs. Sacramento Kings         NaN   
2397      Home                  Utah Jazz vs. Sacramento Kings         NaN   
2398      Away          San Antonio Spurs vs. Dallas Mavericks         NaN   
2399      Home          San Antonio Spurs vs. Dallas Mavericks         NaN   

         TEAM_ID TEAM_ABBREVIATION  TEAM_ID_OPP  YEAR  MONTH  D

In [6]:

def replace_team_names_with_abbreviations(row):
    matchup_str = row['MATCHUP']
    for team, abbreviation in team_to_abbreviation.items():
        matchup_str = matchup_str.replace(team, abbreviation)
    
    # If the team is Away, replace "vs." with "@"
    if row['Home_Away'] == 'Away':
        matchup_str = matchup_str.replace(" vs. ", " @ ")
    return matchup_str


final_data['MATCHUP'] = final_data.apply(replace_team_names_with_abbreviations, axis=1)
#print(final_data)
print(len(final_data))
# Create the unique matchup ID
def create_matchup_id(matchup):
    # Split the teams based on " vs. " or " @ "
    teams = matchup.split(' vs. ') if ' vs. ' in matchup else matchup.split(' @ ')
    # Sort the teams alphabetically and concatenate
    return ''.join(sorted(teams))

final_data['MATCHUP_ID'] = final_data['MATCHUP'].apply(create_matchup_id)
print(final_data.head())
print(len(final_data))


2400
  Home_Away      MATCHUP  WL_encoded     TEAM_ID TEAM_ABBREVIATION  \
0      Away    GSW @ NOP         NaN  1610612740               NOP   
1      Away    POR @ HOU         NaN  1610612745               HOU   
2      Home  GSW vs. NOP         NaN  1610612744               GSW   
3      Home  POR vs. HOU         NaN  1610612757               POR   
4      Away    LAC @ UTA         NaN  1610612762               UTA   

   TEAM_ID_OPP  YEAR  MONTH  DAY MATCHUP_ID  
0   1610612744  2024      4   12     GSWNOP  
1   1610612757  2024      4   12     HOUPOR  
2   1610612740  2024      4   12     GSWNOP  
3   1610612745  2024      4   12     HOUPOR  
4   1610612746  2024      4   12     LACUTA  
2400


In [7]:
# Taking the weeks worth of games
# Create a Date column in the DataFrame
final_data['Date'] = pd.to_datetime(final_data[['YEAR', 'MONTH', 'DAY']])

#***making into a 1 row per game format***
#get the first matchup_id per date
#final_data = final_data.groupby(['MATCHUP_ID', 'Date']).first().reset_index()


# Get today's date
today = pd.Timestamp.today()-pd.Timedelta(days=1)

# Get the date for one week from today
week_out = today + pd.Timedelta(days=7)

# Filter final_data for dates from today up to one week from now
upcoming_games = final_data[(final_data['Date'] >= today) & (final_data['Date'] <= week_out)] # 
upcoming_games.reset_index(drop=True, inplace=True)
#sort by date
upcoming_games.sort_values(by=['Date'], inplace=True)
print(upcoming_games)

# Filter data for dates before tomorrow and save it as old_data
today = pd.Timestamp.today()+pd.Timedelta(days=1)
old_data = final_data#[final_data['Date'] <= today]

#rename WL_encoded to ACTUAL_RESULT 
old_data.rename(columns={'WL_encoded': 'ACTUAL_RESULT'}, inplace=True)
#add a prediction column full of nan
#old_data['PREDICTION'] = np.nan
#print(old_data)
old_data.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\23_24_season_games_past.csv', index=False)

# Drop the 'Date' column
#final_data.drop('Date', axis=1, inplace=True)
#upcoming_games.drop('Date', axis=1, inplace=True)
#old_data.drop('Date', axis=1, inplace=True)

#make predictions on 23_24_season_games_clean (upcoming games with previous season averages)
#saving plan: make predictions on 23_24_season_games_clean (upcoming games with previous season averages)
#1. get the weeks worth of upcoming games from today on so we can't predict on previous games
#2. filter the old data for games including today (so we can add the predictions to this data and the actual results will get added the next day without changing the prediction based on new data)
#3. merge the old data with the predictions data to save yesterday's results with the prediction attached
#4. old_data will have today's games and the predictions will be input onto that data, 
# yesterday's games will have the actual results and will update just that because the model can only predict on today's games moving forward (upcoming_games)
#5. the predictions will be saved to a csv file 

   Home_Away      MATCHUP  WL_encoded     TEAM_ID TEAM_ABBREVIATION  \
33      Home  WAS vs. PHI         NaN  1610612764               WAS   
23      Home  DAL vs. OKC         NaN  1610612742               DAL   
22      Away    DAL @ OKC         NaN  1610612760               OKC   
21      Home  POR vs. NOP         NaN  1610612757               POR   
20      Away    POR @ NOP         NaN  1610612740               NOP   
..       ...          ...         ...         ...               ...   
87      Home  UTA vs. LAL         NaN  1610612762               UTA   
46      Away    POR @ MIN         NaN  1610612750               MIN   
47      Home  POR vs. MIN         NaN  1610612757               POR   
48      Away    MEM @ MIL         NaN  1610612749               MIL   
49      Home  MEM vs. MIL         NaN  1610612763               MEM   

    TEAM_ID_OPP  YEAR  MONTH  DAY MATCHUP_ID       Date  
33   1610612755  2024      2   10     PHIWAS 2024-02-10  
23   1610612760  2024      2   

C:\Users\ghadf\AppData\Local\Temp\ipykernel_14844\3973936647.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games.sort_values(by=['Date'], inplace=True)


In [8]:
# Read in the data for averages from X after it's dropped the columns we don't need and just before preprocessing through encoding
#on a long short-term basis

future_season_data_stats = pd.read_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\future_season_data_stats.csv')
#print(future_season_data_stats.head())
#drop WL_encoded column
#future_season_data_stats.drop(['WL_encoded', 'TEAM_ABBREVIATION', 'MATCHUP', 'YEAR', 'MONTH', 'DAY'], axis=1, inplace=True)

#print(future_season_data_stats.head())
print(len(future_season_data_stats))
print(future_season_data_stats.columns)


1722
Index(['TEAM_ID', 'Home_Away', 'MATCHUP_ID', 'PTS', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FTM', 'FTA', 'AST', 'OREB', 'TOV', 'STL', 'BLK', 'REB', 'MIN',
       'DREB', 'ORtg_Oliver', 'DRtg_Oliver', 'PTS_OPP', 'FGM_OPP', 'FGA_OPP',
       'FG3M_OPP', 'FG3A_OPP', 'FTM_OPP', 'FTA_OPP', 'AST_OPP', 'OREB_OPP',
       'DREB_OPP', 'TOV_OPP', 'STL_OPP', 'BLK_OPP', 'REB_OPP', 'MIN_OPP',
       'ORtg_Oliver_OPP', 'DRtg_Oliver_OPP', 'PTS_PER_MIN', 'FG_PCT',
       'FG3_PCT', 'FT_PCT', 'TS%', 'eFG%', 'AST%', 'Offensive_Possessions',
       'ORtg', 'PER%', 'OFF_EFF', 'PTS_PER_MIN_OPP', 'FG_PCT_OPP',
       'FG3_PCT_OPP', 'FT_PCT_OPP', 'TS%_OPP', 'eFG%_OPP', 'AST%_OPP',
       'Defensive_Possessions', 'DRtg', 'DPER%', 'PTS_DIFF',
       'PTS_PER_MIN_DIFF', 'FG_PCT_DIFF', 'FG3_PCT_DIFF', 'FT_PCT_DIFF',
       'TS%_DIFF', 'eFG%_DIFF', 'AST%_DIFF', 'ORtg_DIFF', 'PER%_DIFF',
       'ORtg_Oliver_DIFF', 'DRtg_Oliver_DIFF'],
      dtype='object')


In [9]:

print("upcoming games= ",upcoming_games.head())
print(len(upcoming_games))
print("future_season_data",future_season_data_stats.head())
print(len(future_season_data_stats))
# Merge final_data with future_season_data_stats on TEAM_ID and Home_Away because we want to see the averages by home and away attached for the models
combined_data = pd.merge(upcoming_games, future_season_data_stats, on=['TEAM_ID', 'Home_Away', 'MATCHUP_ID'], how='left') #, 'MATCHUP'
#print(combined_data.head())

#count nan values
#print(combined_data.isnull().sum())


#view the team_id columns that have nan values
#print(combined_data[combined_data['FG3A'].isnull()])

# drop  Home_Away_x, MATCHUP_x, TEAM_ABBREVIATION_x and change any columns that end with _y to not have _y at the end
#combined_data = combined_data.rename(columns={'Home_Away_y': 'Home_Away', 'MATCHUP_y': 'MATCHUP'})

# drop the team_abbreviaton column
combined_data.drop(['TEAM_ABBREVIATION'], axis=1, inplace=True)
#print(combined_data.head())

# matchup unique values
#print(combined_data['MATCHUP'].unique())
#drop matchup
combined_data.drop(['MATCHUP'], axis=1, inplace=True)

#print length of combined_data
#print(len(combined_data))

feature_order = ['PTS_PER_MIN', 'PTS_DIFF', 'PTS_PER_MIN_DIFF','TEAM_ID', 'TEAM_ID_OPP', 'FG_PCT', 'FG3_PCT', 'FT_PCT',
                  'PLUS_MINUS', 'Home_Away', 'MATCHUP_ID',# 'FG_PCT_OPP', 'FG3_PCT_OPP',  'SEASON_ID', 'GAME_ID'
                   'TS%', 'ORtg', 'PER%', 'eFG%', 'AST%', #'FT_PCT_OPP', 'PLUS_MINUS_OPP','TS%_OPP', 'eFG%_OPP', 'AST%_OPP', , 'MATCHUP'
                  'YEAR', 'MONTH', 'DAY', #'DRtg', 'DPER%',
                 'FG_PCT_DIFF','FG3_PCT_DIFF','FT_PCT_DIFF','TS%_DIFF','eFG%_DIFF','AST%_DIFF','ORtg_DIFF','PER%_DIFF'] #, 'MATCHUP'

categorical_features = [ 'TEAM_ID', 'TEAM_ID_OPP', 'Home_Away', 'MATCHUP_ID'] 

# Reorder columns in the new_data DataFrame
#combined_data_ordered = combined_data[feature_order]
#print(combined_data.head())

#filter for only today's games
today = pd.Timestamp.today()
combined_data = combined_data[combined_data['YEAR'] == today.year]
combined_data = combined_data[combined_data['MONTH'] == today.month]
combined_data = combined_data[combined_data['DAY'] == today.day]
print(combined_data.shape)
#print(combined_data.columns)

# Write the data to a CSV file
combined_data.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\23_24_season_games_clean.csv', index=False)


upcoming games=     Home_Away      MATCHUP  WL_encoded     TEAM_ID TEAM_ABBREVIATION  \
33      Home  WAS vs. PHI         NaN  1610612764               WAS   
23      Home  DAL vs. OKC         NaN  1610612742               DAL   
22      Away    DAL @ OKC         NaN  1610612760               OKC   
21      Home  POR vs. NOP         NaN  1610612757               POR   
20      Away    POR @ NOP         NaN  1610612740               NOP   

    TEAM_ID_OPP  YEAR  MONTH  DAY MATCHUP_ID       Date  
33   1610612755  2024      2   10     PHIWAS 2024-02-10  
23   1610612760  2024      2   10     DALOKC 2024-02-10  
22   1610612742  2024      2   10     DALOKC 2024-02-10  
21   1610612740  2024      2   10     NOPPOR 2024-02-10  
20   1610612757  2024      2   10     NOPPOR 2024-02-10  
88
future_season_data       TEAM_ID Home_Away MATCHUP_ID         PTS        FGM   FGA       FG3M  \
0  1610612737      Away     ATLBKN  111.500000  42.500000  92.0   9.000000   
1  1610612737      Away     AT